In [5]:
test_ds = "../../jl_fs/acg_data/test"
checkpoint_path = "../../jl_fs/chkpoints/run3/model_save_31.pth"


In [2]:
!nvidia-smi

Thu Dec  5 21:14:51 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.4     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA RTX A5000               On  | 00000000:3D:00.0 Off |                  Off |
| 30%   28C    P8              25W / 230W |      1MiB / 24564MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [38]:
import argparse

parser = argparse.ArgumentParser(description="Train")
parser.add_argument("--train_feature_root", type=str, default="../data/pre_3/train")
parser.add_argument("--val_feature_root", type=str, default="../data/pre_3/val")
parser.add_argument("--window", type=float, default=15)



parser.add_argument("--tokenizer_name", type=str, default="meta-llama/Meta-Llama-3-8B")
parser.add_argument("--model_id", type=str, default="meta-llama/Meta-Llama-3-8B")


# parser.add_argument("--tokenizer_name", type=str, default="facebook/opt-350m")
# parser.add_argument("--model_id", type=str, default="facebook/opt-350m")
parser.add_argument("--max_token_length", type=int, default=128)
parser.add_argument("--train_ann_root", type=str, default="./dataset/MatchTime/train")
parser.add_argument("--train_batch_size", type=int, default=8)
parser.add_argument("--train_num_workers", type=int, default=4)


parser.add_argument("--val_batch_size", type=int, default=8)
parser.add_argument("--val_num_workers", type=int, default=4)


parser.add_argument("--lr", type=float, default=1e-4)
parser.add_argument("--num_epoch", type=int, default=10)
parser.add_argument("--num_query_tokens", type=int, default=32)
parser.add_argument("--num_video_query_token", type=int, default=32)
parser.add_argument("--num_features", type=int, default=1024)
parser.add_argument("--fps", type=int, default=1)
parser.add_argument("--model_output_dir", type=str, default="../ckpt")
parser.add_argument("--device", type=str, default="cuda")
parser.add_argument("--qbit_4", type=bool, default=True)



# If continue training from any epoch
parser.add_argument("--continue_train", type=bool, default=False)
parser.add_argument("--pre_max_CIDEr", type=float, default=0.0)
parser.add_argument("--pre_epoch", type=int, default=0)
parser.add_argument("--load_ckpt", type=str, default="./ckpt/model_save_best_val_CIDEr.pth")


args = parser.parse_args()

usage: ipykernel_launcher.py [-h] [--train_feature_root TRAIN_FEATURE_ROOT]
                             [--val_feature_root VAL_FEATURE_ROOT]
                             [--window WINDOW]
                             [--tokenizer_name TOKENIZER_NAME]
                             [--model_id MODEL_ID]
                             [--max_token_length MAX_TOKEN_LENGTH]
                             [--train_ann_root TRAIN_ANN_ROOT]
                             [--train_batch_size TRAIN_BATCH_SIZE]
                             [--train_num_workers TRAIN_NUM_WORKERS]
                             [--val_batch_size VAL_BATCH_SIZE]
                             [--val_num_workers VAL_NUM_WORKERS] [--lr LR]
                             [--num_epoch NUM_EPOCH]
                             [--num_query_tokens NUM_QUERY_TOKENS]
                             [--num_video_query_token NUM_VIDEO_QUERY_TOKEN]
                             [--num_features NUM_FEATURES] [--fps FPS]
                        

SystemExit: 2

In [3]:

from dataset import ACGDataset
from optmodel import OPTModel
from torch.utils.data import DataLoader
from tqdm import tqdm
from llamamodel import LLAMAModel
from transformers import AdamW
import torch
import numpy as np
import random
import os
from pycocoevalcap.cider.cider import Cider



In [6]:

test_ds = ACGDataset(root_dir=test_ds,
                               tokenizer_name="meta-llama/Meta-Llama-3-8B", 
                               max_token_length=128)

/root/miniconda3/envs/py3.10/lib/python3.10/site-packages/transformers/models/auto/tokenization_auto.py:786: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(


In [7]:
from llamamodel import LLAMAModel

In [8]:
model = LLAMAModel("meta-llama/Meta-Llama-3-8B", "meta-llama/Meta-Llama-3-8B",
                        num_query_tokens=32 ,
                        num_video_query_token=32,
                        num_features=1024, device='cuda', qbit_4= True)

`low_cpu_mem_usage` was None, now set to True since model is quantized.


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [9]:
other_parts_state_dict = torch.load(checkpoint_path)
    
missing_keys, unexpected_keys = model.load_state_dict(other_parts_state_dict,strict=False)

/tmp/ipykernel_3678/403821139.py:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  other_parts_state_dict = torch.load(checkpoint_path)


In [10]:
import re

def remove_special_token(text):
    """
    Removes the special token <|begin_of_text|> from the given text.

    Args:
        text (str): The input string containing the special token.

    Returns:
        str: The cleaned string without the special token.
    """
    # Define the regex pattern to match <|begin_of_text|>
    pattern = r"<\|begin_of_text\|>"
    
    # Use re.sub to replace the special token with an empty string
    cleaned_text = re.sub(pattern, "", text)
    
    # Strip any leading or trailing whitespace after removal
    return cleaned_text.strip()


In [78]:
batch_in = test_ds.collator([test_ds[234], test_ds[122] , test_ds[10]])

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


In [11]:

test_data_loader = DataLoader(test_ds, batch_size=4, num_workers=2, drop_last=False, shuffle=False, pin_memory=True, collate_fn=test_ds.collator)

model.eval() 
p,t = model(batch_in,validating=True) #p-> predictions, t-> truth
p =[remove_special_token(text) for text in p]
vid_ids = batch_in['vid_ids']

NameError: name 'batch_in' is not defined

In [12]:
from datasets import Dataset, DatasetDict
from torch.utils.data import DataLoader

# Initialize DataLoader
test_data_loader = DataLoader(
    test_ds, batch_size=2, num_workers=1, drop_last=False, shuffle=False,
    pin_memory=True, collate_fn=test_ds.collator
)

# Switch model to evaluation mode
model.eval()

# Initialize storage for results
results = []

# Iterate through the test dataset
for batch_in in test_data_loader:
    try:
        # Perform inference
        p, t = model(batch_in, validating=True)
        
        # Remove special tokens from predictions
        p = [remove_special_token(text) for text in p]
        
        # Extract video IDs
        vid_ids = batch_in['vid_ids']
        
        # Collect results
        for vid_id, pred, truth in zip(vid_ids, p, t):
            results.append({"vid_id": vid_id, "prediction": pred, "truth": truth})

    except Exception as e:
        print(e)
# Create a Hugging Face dataset
results_dataset = Dataset.from_dict(results)

# Push the dataset to the Hugging Face Hub
results_dataset.push_to_hub("Aadharsh/ACG-test-36pth", private=True)


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
The attention m

AttributeError: 'list' object has no attribute 'items'

In [13]:
results_dataset = Dataset.from_list(results)

In [14]:
results_dataset = Dataset.from_list(results)

# Push the dataset to the Hugging Face Hub
results_dataset.push_to_hub("Aadharsh/ACG-test-31pth", private=True)


Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

CommitInfo(commit_url='https://huggingface.co/datasets/Aadharsh/ACG-test-31pth/commit/6f7258d98ff66dd1d96da587ad100273119e6644', commit_message='Upload dataset', commit_description='', oid='6f7258d98ff66dd1d96da587ad100273119e6644', pr_url=None, pr_revision=None, pr_num=None)

In [99]:
results_dataset[0]

{'vid_id': 'CricketVideo8_video_segment_7',
 'prediction': 'The delivery is heading towards the batsman.',
 'truth': '[BATSMAN] hits a shot to the third man boundary, scoring four runs.'}

In [16]:
import json

# Save the results to a JSON file
with open("results31.json", "w") as file:
    json.dump(results, file, indent=4)


In [1]:
!nvidia-smi

Thu Dec  5 22:25:29 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.4     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA RTX A5000               On  | 00000000:3D:00.0 Off |                  Off |
| 30%   26C    P5              32W / 230W |      1MiB / 24564MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [ ]:
import os
import json
import re
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
import torch
from tqdm import tqdm

# Function to extract assistant's response
def extract_assistant_response(text):
    """
    Extracts the assistant's response using regex.
    The response is located between <|start_header_id\|>assistant<|end_header_id\|> and <|eot_id\|>.
    """
    pattern = r"<\|start_header_id\|>assistant<\|end_header_id\|>(.*?)<\|eot_id\|>"
    match = re.search(pattern, text, re.DOTALL)
    return match.group(1).strip() if match else ""

# Model and tokenizer setup
model_id = "meta-llama/Llama-3.1-8B-Instruct"
nf4_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=torch.bfloat16,
)

model_nf4 = AutoModelForCausalLM.from_pretrained(model_id, quantization_config=nf4_config)

tokenizer = AutoTokenizer.from_pretrained(model_id, padding_side='left')
tokenizer.pad_token = tokenizer.eos_token
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Prompt for the model
prompt = """
You will be provided with cricket commentary. Your task is to standardize the commentary by rephrasing it into a formal and concise tone. Remove personal identifiers and replace them with the following special tokens: [BATSMAN], [BOWLER], [FIELDER], [UMPIRE], [VENUE]. Ensure that the commentary remains concise and retains its original meaning.

If no commentary or irrelevant text is provided, output "None".
"""

# Function to load and process JSON files
def load_commentaries_from_json(folder_path):
    """
    Load and combine the `text` fields from all JSON files in a folder.
    Returns a list of (filename, commentary) pairs.
    """
    all_commentaries = []
    filenames=[]
    for file_name in os.listdir(folder_path):
        if file_name.endswith(".json"):
            filenames.append(file_name.split(".")[0])
            file_path = os.path.join(folder_path, file_name)
            with open(file_path, "r") as f:
                data = json.load(f)
                # Combine all `text` fields to form one commentary
                commentary = " ".join(item["text"].strip() for item in data if "text" in item)
                all_commentaries.append((file_name, commentary))
    return all_commentaries,filenames

# Function to perform batch inference
import os

# def batch_inference(commentaries, filenames, dest_dir, batch_size=4):
#     """
#     Perform inference in batches and save responses to text files.
#     """
#     results = []
#     for i in tqdm(range(0, len(commentaries), batch_size)):
#         batch = commentaries[i : i + batch_size]
#         batch_filenames = filenames[i:i + batch_size]
        
#         # Prepare batch messages
#         messages_batch = [
#             [{"role": "system", "content": prompt}, {"role": "user", "content": f"Commentary: {c[1]}"}]
#             for c in batch
#         ]

#         # Prepare inputs for the model
#         text_batch = tokenizer.apply_chat_template(messages_batch, tokenize=False)
#         inputs_batch = tokenizer(text_batch, return_tensors="pt", padding=True)
#         inputs_batch = {key: tensor.to(device) for key, tensor in inputs_batch.items()}

#         # Generate responses
#         generated_ids_batch = model_nf4.generate(
#             **inputs_batch,
#             max_new_tokens=150,
#             do_sample=True,
#             temperature=0.7,
#             pad_token_id=tokenizer.eos_token_id
#         )
        
#         # Decode and extract responses
#         outputs = tokenizer.batch_decode(generated_ids_batch, skip_special_tokens=False)
#         parsed_outs = [extract_assistant_response(output) for output in outputs]

#         # Save each response to the corresponding file in dest_dir
#         for j, parsed_out in enumerate(parsed_outs):
#             filename = batch_filenames[j]
#             file_path = os.path.join(dest_dir, f"{filename}.txt")
#             with open(file_path, "w") as f:
#                 f.write(parsed_out)

#         # Add filename and response pair to results
#         results.extend(zip([c[0] for c in batch], parsed_outs))
    
#     return results

def batch_inference(commentaries, filenames, dest_dir, batch_size=4):
    """
    Perform inference in batches and save responses to text files. Skip the entire batch if all files already exist.
    """
    results = []
    for i in tqdm(range(0, len(commentaries), batch_size)):
        batch = commentaries[i : i + batch_size]
        batch_filenames = filenames[i:i + batch_size]
        
        # Check if the output files already exist for the batch, and skip the entire batch if so
        existing_files = [
            os.path.join(dest_dir, f"{filename}.txt") for filename in batch_filenames if os.path.exists(os.path.join(dest_dir, f"{filename}.txt"))
        ]
        
        # If all files in the batch exist, skip the entire batch
        if len(existing_files) == len(batch_filenames):
            print(f"Skipping batch {i // batch_size + 1} as all files already exist: {', '.join(existing_files)}")
            continue
        
        # Prepare batch messages
        messages_batch = [
            [{"role": "system", "content": prompt}, {"role": "user", "content": f"Commentary: {c[1]}"}]
            for c in batch
        ]

        # Prepare inputs for the model
        text_batch = tokenizer.apply_chat_template(messages_batch, tokenize=False)
        inputs_batch = tokenizer(text_batch, return_tensors="pt", padding=True)
        inputs_batch = {key: tensor.to(device) for key, tensor in inputs_batch.items()}

        # Generate responses
        generated_ids_batch = model_nf4.generate(
            **inputs_batch,
            max_new_tokens=150,
            do_sample=True,
            temperature=0.7,
            pad_token_id=tokenizer.eos_token_id
        )
        
        # Decode and extract responses
        outputs = tokenizer.batch_decode(generated_ids_batch, skip_special_tokens=False)
        parsed_outs = [extract_assistant_response(output) for output in outputs]

        # Save each response to the corresponding file in dest_dir
        for j, parsed_out in enumerate(parsed_outs):
            filename = batch_filenames[j]
            file_path = os.path.join(dest_dir, f"{filename}.txt")

            # Skip saving if the file already exists
            if os.path.exists(file_path):
                continue  # Already skipped above, but included for safety

            # Save the response to the file
            with open(file_path, "w") as f:
                f.write(parsed_out)

        # Add filename and response pair to results
        results.extend(zip([c[0] for c in batch], parsed_outs))
    
    return results



# Main function
def process_directory_and_run_inference(folder_path, destination_dir, batch_size=4):
    """
    Process a directory of JSON files, run inference, and save the results.
    """
    # Ensure destination directory exists
    os.makedirs(destination_dir, exist_ok=True)

    # Load all commentaries with filenames
    commentaries,filenames = load_commentaries_from_json(folder_path)

    # Run batch inference
    results = batch_inference(commentaries,filenames, destination_dir, batch_size=batch_size)

    # # Save results to destination directory
    # for file_name, response in results:
    #     output_file_path = os.path.join(destination_dir, file_name.replace(".json", ".txt"))
    #     with open(output_file_path, "w") as f:
    #         f.write(response)
    #     print(f"Processed {file_name} -> Saved to {output_file_path}")

# Example usage
folder_path = "/home/ACG/data/raw/whisperx_transcripts"  # Replace with your JSON folder path
destination_dir = "/home/ACG/data/pre_2/standardized_transcripts"  # Replace with your destination folder path
process_directory_and_run_inference(folder_path, destination_dir, batch_size=2)
